In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_excel(r"C:\Users\usama\Downloads\insurance.xlsx")
df.head()


,age,sex,bmi,children,smoker,region,expenses
0,19.0,female,27.9,0.0,yes,southwest,16884.92
1,18.0,male,33.8,1.0,no,southeast,1725.55
2,28.0,male,33.0,3.0,no,southeast,4449.46
3,33.0,male,22.7,0.0,no,northwest,21984.47
4,32.0,male,28.9,0.0,no,northwest,3866.86


In [3]:
df.shape

(1338, 7)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   float64
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   float64
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   expenses  1338 non-null   float64
dtypes: float64(4), object(3)
memory usage: 73.3+ KB


In [5]:
categorical=[]
continous=[]
check=[]
d_types=dict(df.dtypes)
for name, type in d_types.items():
    if str(type)== 'object':
        categorical.append(name)
    elif str(type)== 'float64':
        continous.append(name)
    else:
        check.append(name)
print(categorical)
print(continous)
print(check)

['sex', 'smoker', 'region']
['age', 'bmi', 'children', 'expenses']
[]


In [6]:
continous=['bmi', 'age', 'expenses']
discrete=['sex','smoker','region','children']

In [7]:
df[continous].describe()

,bmi,age,expenses
count,1338.000000,1338.000000,1338.000000
mean,30.665471,39.207025,13270.422414
std,6.098382,14.049960,12110.011240
min,16.000000,18.000000,1121.870000
25%,26.300000,27.000000,4740.287500
50%,30.400000,39.000000,9382.030000
75%,34.700000,51.000000,16639.915000
max,53.100000,64.000000,63770.430000


In [8]:
df[continous].corr()

,bmi,age,expenses
bmi,1.000000,0.109341,0.198576
age,0.109341,1.000000,0.299008
expenses,0.198576,0.299008,1.000000


In [9]:
df.isnull().sum()

age         0
sex         0
bmi         0
children    0
smoker      0
region      0
expenses    0
dtype: int64

In [10]:
df.drop('region',axis=1,inplace=True)

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
df['sex'].unique()

array(['female', 'male'], dtype=object)

In [13]:
df['smoker'].unique()

array(['yes', 'no'], dtype=object)

In [14]:
df['sex'].replace({'female':0 ,'male':1}, inplace=True)
df['smoker'].replace({'no':0 ,'yes':1}, inplace=True)

In [15]:
df

,age,sex,bmi,children,smoker,expenses
0,19.0,0,27.9,0.0,1,16884.92
1,18.0,1,33.8,1.0,0,1725.55
2,28.0,1,33.0,3.0,0,4449.46
3,33.0,1,22.7,0.0,0,21984.47
4,32.0,1,28.9,0.0,0,3866.86
...,...,...,...,...,...,...
1333,50.0,1,31.0,3.0,0,10600.55
1334,18.0,0,31.9,0.0,0,2205.98
1335,18.0,0,36.9,0.0,0,1629.83
1336,21.0,0,25.8,0.0,0,2007.95


In [16]:
X=df.drop('expenses', axis=1)
y=df['expenses']

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=9)

In [18]:
#lasso regression with defailt parameters

from sklearn.linear_model import Lasso
lasso_base=Lasso()
lasso_base.fit(X_train,y_train)
print('intercept:', lasso_base.intercept_)
print('coef:', lasso_base.coef_)

train_prediction= lasso_base.predict(X_train)
test_prediction=lasso_base.predict(X_test)

print('R2_train:', lasso_base.score(X_train, y_train))
print('R2_test:', lasso_base.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
print('cross validation score:', cross_val_score(lasso_base,X,y,cv=5).mean())


intercept: -12153.909094452427
coef: [  264.54965338  -117.09531679   318.45932552   411.37847149
 23981.56471336]
R2_train: 0.7593835903914579
R2_test: 0.7006013036702504
cross validation score: 0.7465344512602905


In [19]:
#hyperparameter tuning
from sklearn.model_selection import GridSearchCV

#model
estimator= Lasso()

#parameter
param_grid={'alpha':list(range(1,70))}
 #identifying the best value of the parameter within given values for the given data

model_hp=GridSearchCV( estimator,param_grid,cv=5,scoring='r2')
model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 60}

In [20]:
# rebuild lasso regression with best parameters

from sklearn.linear_model import Lasso
lasso_base=Lasso(alpha=60)
lasso_base.fit(X_train,y_train)
print('intercept:', lasso_base.intercept_)
print('coef:', lasso_base.coef_)

train_prediction= lasso_base.predict(X_train)
test_prediction=lasso_base.predict(X_test)

print('R2_train:', lasso_base.score(X_train, y_train))
print('R2_test:', lasso_base.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
print('cross validation score:', cross_val_score(lasso_base,X,y,cv=5).mean())


intercept: -12045.192067679147
coef: [  264.37213484    -0.           317.0408608    373.19599922
 23621.90440143]
R2_train: 0.7592042059376202
R2_test: 0.7008928970688789
cross validation score: 0.7466813254014273


In [21]:
#the second coef is zero which means the gender feature is not an important column

In [22]:
X=X.drop(X.columns[[1]] ,axis=1)
y=df['expenses']
X_train, X_test, y_train,y_test=train_test_split(X,y,test_size=0.2, random_state=9)



from sklearn.linear_model import Lasso
lasso_base=Lasso(alpha=60)
lasso_base.fit(X_train,y_train)
print('intercept:', lasso_base.intercept_)
print('coef:', lasso_base.coef_)

train_prediction= lasso_base.predict(X_train)
test_prediction=lasso_base.predict(X_test)

print('R2_train:', lasso_base.score(X_train, y_train))
print('R2_test:', lasso_base.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
print('cross validation score:', cross_val_score(lasso_base,X,y,cv=5).mean())


intercept: -12045.187463841945
coef: [  264.37194096   317.04095573   373.19607238 23621.90427308]
R2_train: 0.7592042058163877
R2_test: 0.7008929179833459
cross validation score: 0.7466852748024613


# ridge regression

In [93]:
#pd.det_option('display.max_rows',5000)

In [18]:
#ridge regression with defailt parameters

from sklearn.linear_model import Ridge
ridge_base=Ridge()
ridge_base.fit(X_train,y_train)
print('intercept:', ridge_base.intercept_)
print('coef:', ridge_base.coef_)

train_prediction= ridge_base.predict(X_train)
test_prediction=ridge_base.predict(X_test)

print('R2_train:', ridge_base.score(X_train, y_train))
print('R2_test:', ridge_base.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
print('cross validation score:', cross_val_score(ridge_base,X,y,cv=5).mean())


intercept: -12131.383174500303
coef: [  264.4786592   -112.37962155   318.56350557   413.12069122
 23853.85951773]
R2_train: 0.7593639632162803
R2_test: 0.7008629672692219
cross validation score: 0.746520309627156


In [34]:
from sklearn.model_selection import GridSearchCV

#model
estimator= Ridge()

#parameter
param_grid={'alpha':(l)}
 #identifying the best value of the parameter within given values for the given data

model_hp=GridSearchCV( estimator,param_grid,cv=5,scoring='r2')
model_hp.fit(X_train,y_train)
model_hp.best_params_

{'alpha': 0.8999999999999999}

In [35]:

# rebuild lasso regression with best parameters
from sklearn.linear_model import Ridge
ridge_base=Ridge(0.8999999999999999)
ridge_base.fit(X_train,y_train)
print('intercept:', ridge_base.intercept_)
print('coef:', ridge_base.coef_)

train_prediction= ridge_base.predict(X_train)
test_prediction=ridge_base.predict(X_test)

print('R2_train:', ridge_base.score(X_train, y_train))
print('R2_test:', ridge_base.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
print('cross validation score:', cross_val_score(ridge_base,X,y,cv=5).mean())



intercept: -12133.707814265415
coef: [  264.48578495  -113.29390313   318.5564332    413.01435521
 23867.19630345]
R2_train: 0.759367688924302
R2_test: 0.7008380259471574
cross validation score: 0.7465228120563532


#  Elastic NET

In [18]:
#ridge regression with defailt parameters

from sklearn.linear_model import ElasticNet
enr_base=ElasticNet()
enr_base.fit(X_train,y_train)
print('intercept:', enr_base.intercept_)
print('coef:', enr_base.coef_)

train_prediction= enr_base.predict(X_train)
test_prediction=enr_base.predict(X_test)

print('R2_train:', enr_base.score(X_train, y_train))
print('R2_test:', enr_base.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
print('cross validation score:', cross_val_score(enr_base,X,y,cv=5).mean())


intercept: -8521.551099539573
coef: [ 254.9425317   354.21968666  328.21030534  444.72779674 6012.2843481 ]
R2_train: 0.4040370376357658
R2_test: 0.36842833194908864
cross validation score: 0.3887681232987844


In [20]:
from sklearn.model_selection import GridSearchCV

#model
estimator= ElasticNet()

#parameter
param_grid={'alpha':list(range(1,100)),
        'l1_ratio': [0.1,0.2,0.3,0.4,0.5,0.75,0.9,0.95,1]}
 #identifying the best value of the parameter within given values for the given data

model_hp=GridSearchCV( estimator,param_grid,cv=5,scoring='r2')
model_hp.fit(X_train,y_train)
model_hp.best_params_

# A 1 for lasso means it's the best

{'alpha': 60, 'l1_ratio': 1}

In [21]:
from sklearn.linear_model import ElasticNet
enr_base=ElasticNet(alpha=60, l1_ratio=1)
enr_base.fit(X_train,y_train)
print('intercept:', enr_base.intercept_)
print('coef:', enr_base.coef_)

train_prediction= enr_base.predict(X_train)
test_prediction=enr_base.predict(X_test)

print('R2_train:', enr_base.score(X_train, y_train))
print('R2_test:', enr_base.score(X_test, y_test))

from sklearn.model_selection import cross_val_score
print('cross validation score:', cross_val_score(enr_base,X,y,cv=5).mean())


intercept: -12045.192067679147
coef: [  264.37213484    -0.           317.0408608    373.19599922
 23621.90440143]
R2_train: 0.7592042059376202
R2_test: 0.7008928970688789
cross validation score: 0.7466813254014273
